## 使用SageMaker的BYOS(Bring Your Oen Script)方式部署大语言模型

利用SageMaker部署大语言模型的原理如下：

<img src="imgs/sagemaker_deploy_model.jpg" style="width: 850px;"></img>

SageMaker提供了几种不同的Container来部署模型，[DJL](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials-deepspeed-djl.html)是AWS提供的用于部署大模型的容器。使用它，我们可以使用DeepSpeed、HuggingFace、FatserTransformer来进行加速。

<img src="imgs/djl.jpg" style="width: 850px;"></img>

目前不同的加速框架支持的模型如下（2023年3月）：

<img src="imgs/djl_models.jpg" style="width: 850px;"></img>

目前DeepSpeed已经支持LLaMA，并且性能提升达到3倍。

### 可部署的模型

使用DJL容器部署模型，对于一部分支持的模型，只需要通过定义DeepSpeedModel 或 HuggingFaceAccelerateModel 来部署即可。可直接部署的模型有：
 * LLaMA
 * LLaMA2系列

这些不需要提供脚本，所需参数都在模型定义里。根据模型不同，可使用的加速框架不同，如huggingface、deepspeed，来定义相应的DeepSpeedModel 或 HuggingFaceAccelerateModel来部署。


### 部署

准备：
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
# 如果需要，更新sagemaker和 aws python sdk boto3
# !pip install --upgrade boto3
# !pip install --upgrade sagemaker

In [1]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
region = sagemaker_session._region_name # region name of the current environment
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
print(bucket)

sagemaker-us-east-1-568765279027


接下来，我们使用Sagemaker进行模型部署。


In [ ]:
    # model_id="shibing624/chinese-alpaca-plus-7b-hf",
    model_id="ziqingyang/chinese-alpaca-2-7b",

In [ ]:
from sagemaker.djl_inference import DeepSpeedModel

instance_type = "ml.g5.2xlarge"
# endpoint_name = 'mt-cn-alpaca-7b-g4dn'
endpoint_name = 'mt-cn-alpaca2-7b-g5'

model = DeepSpeedModel(
    djl_version="0.23.0",
    name="mt-cn-alpaca2-7b-g5-model",
    # model_id="s3://sagemaker-us-east-1-568765279027/mt_models_uploaded/meta-llama--Llama-2-7b-chat-hf",
    # model_id="shibing624/chinese-alpaca-plus-7b-hf",
    model_id="ziqingyang/chinese-alpaca-2-7b",
    role=role,
    number_of_partitions=1,
    max_tokens=4096,
    dtype="fp16",
    low_cpu_mem_usage=True
)


predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=600
)

如果部署过程中出现错误，部署失败，通过下面的方式删除endpoint以及相应的模型。

In [10]:
from sagemaker import serializers, deserializers

del_predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(), 
    deserializer=deserializers.JSONDeserializer()
)
del_predictor.delete_model()
del_predictor.delete_endpoint()


In [ ]:
inputs= [
    {"inputs": "写一首关于交通信号灯的诗"},
    {"inputs": "陨石为什么总能落在陨石坑里?"},
    {"inputs": "为什么爸妈结婚没叫我参加婚礼?"}
]

response = predictor.predict(inputs[0])
# print("\n\n问题: ", inputs[0]["inputs"], "\n回答:\n", response["outputs"])
# response = predictor.predict(inputs[1])
# print("\n\n问题: ", inputs[1]["inputs"], "\n回答:\n", response["outputs"])
# response = predictor.predict(inputs[2])
# print("\n\n问题: ", inputs[2]["inputs"], "\n回答:\n", response["outputs"])

In [ ]:
response

### 通过Sagemaker Endpoint调用
我们已经将模型部署到了Sagemaker Endpoint上，我们就可以通过这个Endpoint名称，来调用模型进行推理，这样即使你停止了这个notebook，也能使用已经部署的模型。

In [6]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint(encoded_json):
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions["outputs"]
    return generated_text



In [7]:
%%time

payload = {"inputs": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案：", "parameters": {"temperature": 0.01}}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint mt-cn-alpaca2-7b-g5 of account 568765279027 not found.

In [19]:
%%time
payload = {"inputs": "天为什么是蓝色的？"}
answer = query_endpoint(json.dumps(payload).encode('utf-8'))
print(answer)
print(len(answer))

天空之所以呈现蓝色,是由于光的散射现象造成的。当太阳光穿过大气层时,光被大气中的分子和小颗粒散射,这些颗粒包括氧气、氮气和水蒸气等分子。这些分子吸收较短波长的光,如紫色和蓝色,而较长波长的光,如红色和橙色,则被分散得更少。

由于蓝色光的波长比红色光短,因此它更容易被分散,而在大气层中被散射的程度也更高,因此在天空的观察中,我们看到了大量的蓝色光。这也是为什么在日落或日出时,太阳光穿过更长的大气层路径,较多的光被散射为红色和橙色,天空呈现出橙色或红色的原因。
231
CPU times: user 3.32 ms, sys: 386 µs, total: 3.71 ms
Wall time: 8.22 s


In [20]:
%%time
payload = {"inputs": "4加2等于几？"}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

4加2等于6。
CPU times: user 3.48 ms, sys: 0 ns, total: 3.48 ms
Wall time: 446 ms


In [21]:
%%time
# ChatGLM支持通过history传递聊天历史
payload = {
    "inputs": "再乘以3呢？",
    "history": [("数学计算：\n3加8等于几？\n答案：", "3加8等于11。")]}
resp_test = query_endpoint(json.dumps(payload).encode('utf-8'))
print(resp_test)

如果你将11乘以3,那么答案是33。
CPU times: user 871 µs, sys: 3.07 ms, total: 3.94 ms
Wall time: 874 ms


### 删除 EndPoint

In [24]:
predictor.delete_model()
predictor.delete_endpoint()